In [2]:
import os 
import string
from numpy import loadtxt
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)
import glob
import numpy as np
from multiprocessing import Pool
import multiprocessing as multi
sys.path.append(os.pardir)
import pandas as pd
import h5py
import re
import gensim


# lines = loadtxt("/home/louis/SharedWindows/edu_data/"+"imdb-edus.train")
# train = []

# with open(dir+"imdb-edus.train", 'r') as train_file:
#     lines = train_file.readlines()


##



# '''text2index'''
# 
# from func import TextDeal
# from func import wrapper_func_four
# 
# vocab = wList
# 
# td = TextDeal()
# load_path = sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/Text/text_list/*.txt' ) )
# name = [ os.path.splitext( os.path.basename( r ) )[0] for r in load_path ]
# save_path = '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/W2I/'



/home/owner/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/owner/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [19]:

##

def wrapper_func_six(tuple_data):
    return(tuple_data[0](tuple_data[1],tuple_data[2],tuple_data[3],tuple_data[4], tuple_data[5], tuple_data[6]) )

def readEduStyledFiles(dir, filename):
    with open(dir+filename, 'r') as train_file:
        lines = train_file.readlines()
    return lines
        
def tokenizeListOfTexts(list, vocab):
    idx = []
    for word in list:
        if word in vocab:
            idx.append(vocab[word].index)
            #print(word)
        # else:
        #     print(word)
    return idx

def addUniqueElementsToList(docs, li):
    for doc in docs:
        for sentence in doc:
            for word in sentence:
                if word not in li:
                    print(word)
                    li.append(word)
    return li

def generateUniqueIndices(path_list):
    vocab_list = []
    for filepath in path_list:
        tmp = np.load(filepath)
        li = addUniqueElementsToList(tmp, vocab_list)
        vocab_list.append(li)
    return vocab_list

def cleanAndTokenizeEDUs(vocab, load_path=None, load_name=None, save_path=None, save_name=None, willSave = False, maxlen = 15):
    
    if ( willSave == True) and (all(v is None for v in [load_path, load_name, save_path, save_name])):
        raise ValueError('Function Arguments cannot all be None Type when saving files')
    elif (willSave == False) and (all(v is None for v in [load_path, load_name])):
        raise ValueError('Load arguments cannot all be None Type when loading file')
    
    lines = readEduStyledFiles(load_path,load_name)

    cleaned_lines = []
    length = len(lines)
    scores = []
    ids = []
    cleaned_doc = []
    cleaned_all = []
    idx_binary = [] # index for binary training only
    vocab_list = []
    doc_count = -1
    
    for i in range(len(lines)):
        cleaned = lines[i].replace('\n','').replace('<s>','').replace("'",'')
        cleaned = text_to_word_sequence(cleaned)
        
        
        if (len(cleaned) == 2): # and (i != 0):
            if  (cleaned[0].isnumeric() and cleaned[1].isnumeric()):
                score = int(cleaned[0])
                id = int(cleaned[1])
                if (score<=4) or (score>=7):
                    idx_binary.append(doc_count) #index for binary classification
                filtered_doc = list(filter(None, cleaned_doc))
                padded_doc = pad_sequences(filtered_doc, maxlen=maxlen)
                cleaned_all.append(padded_doc.tolist()) #add padded text vectors
               
                scores.append(score)
                ids.append(id)
                #vocab_list = addUniqueElementsToList(filtered_doc, vocab_list)
                
                cleaned_doc=[]
                doc_count += 1
                continue
        
        tokenized=tokenizeListOfTexts(cleaned, vocab)
        
        if (i%1000000 == 0) and (i!=0):
            print(str(i)+' out of' + str(length))
            #break
            # print(cleaned_lines[i-500:i])
    
        cleaned_doc.append(tokenized)
        # cleaned_doc = cleaned_doc[0]
   
    del cleaned_all[0]
    del scores[0]
    del idx_binary[0]
    del ids[0]
    
    cleaned_all = np.asarray(cleaned_all)
    scores = np.asarray(scores)
    idx_binary = np.asarray(idx_binary)
    
   
    
    if (willSave == False):
        print('cleanAndTokenizeEDUs Returned and Done')
        return features, labels, idx, ids, vocab_list
    else:
        np.save(save_path + save_name + '_features'+ '.npy', cleaned_all)
        np.save(save_path + save_name + '_labels' + '.npy', scores)
        np.save(save_path + save_name + '_idx_binary_only' + '.npy', idx_binary)
        np.save(save_path + save_name + '_doc_ids' + '.npy', ids)
        print(save_name + ' cleanAndTokenizeEDUs Saved and Done')

dir = '/home/louis/SharedWindows/edu_data/'  



# tokenized = t.fit_on_texts(cleaned)


In [10]:

load_path = '/home/owner/デスクトップ/milnet+edu/data/'  
save_path = '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/'

load_name = ['imdb-edus.train', 'imdb-edus.test','imdb-edus.dev']
save_name = ['train_data','test_data','validation_data']
name_trailers = ['_features','_labels','_idx_binary_only','_doc_ids']
willSave = False
i=0
cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = load_name[i], save_path=save_path,save_name=save_name[i], willSave = False)

[]
cleanAndTokenizeEDUs Returned and Done


(array([], shape=(1, 0, 15), dtype=int32),
 array([], dtype=float64),
 array([-1]),
 [],
 [])

In [20]:
if __name__ == '__main__':
    #data=(関数, 引数)
    isReload = int(input("Type 1 to initiate reload of google word2vec: "))
    
    if isReload:
        print('word2vec model now loading')
        model_word2vec_temp = gensim.models.KeyedVectors.load_word2vec_format('/home/owner/デスクトップ/milnet+edu/GoogleNews-vectors-negative300.bin', binary=True)  
        model_word2vec = model_word2vec_temp

    print('word2vec model loaded')

    load_path = '/home/owner/デスクトップ/milnet+edu/data/'  
    save_path = '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/'
      
    load_name = ['imdb-edus.train', 'imdb-edus.test','imdb-edus.dev']
    save_name = ['train_data','test_data','validation_data']
    name_trailers = ['_features','_labels','_idx_binary_only','_doc_ids']
    willSave = True
    

    #for i in range(len(load_name)):
    #i = 0
    #a,b,c,d = cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = load_name[i], save_path=save_path,save_name=save_name[i], willSave = False)
    
    # features, scores, idx, ids = cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = 'imdb-edus.train', save_path=save_path,save_name=save_name[0], willReturn = False)
    
    vocab = model_word2vec.vocab
    data = [ ( cleanAndTokenizeEDUs, vocab, load_path, load_name[i], save_path, save_name[i] , willSave) for i in range( len(load_name) ) ]
    with Pool( multi.cpu_count()-1 ) as p:
        p.map( wrapper_func_six, data )
    
    print('Starting Vocab List')
    vocab_list = generateUniqueIndices([save_path+name+'_features'+'.npy' for name in save_name])
    
    print('All Done')



Type 1 to initiate reload of google word2vec: 0
word2vec model loaded
1000000 out of11039529
1000000 out of1349298
1000000 out of1335054
2000000 out of11039529
validation_data cleanAndTokenizeEDUs Saved and Done
3000000 out of11039529
test_data cleanAndTokenizeEDUs Saved and Done
4000000 out of11039529
5000000 out of11039529
6000000 out of11039529
7000000 out of11039529
8000000 out of11039529
9000000 out of11039529
10000000 out of11039529
11000000 out of11039529
train_data cleanAndTokenizeEDUs Saved and Done
Starting Vocab List
0
61
2345
4501
10
125
331
200
33
1642
3
165
2040
7276
15
271
11
12373
5
16153
2247
105505
14
1574
30391
86857
4
5036
8409
170
28
194
692
31165
468
8469
15389
2691676
19
67566
1
7026
1143
251
103832
346
888
26
412
550
85
6869
627
350
309
17
4619
94620
126
1966
80
2485
733
7810
1666
1645
68
30
270
1280
87
107
2
131
78949
151
8
164
78
137
159
2652
141
267
598
56
59
15510
353
25131
18
21750
818
452
43
911
15708
95
109
41
1082
278
572747
10704
282
512
3894
2022
449
7

10001
13755
951
4322
18693
48606
92806
2243
2996
998436
4764
23143
770
3339
36124
91664
3427
12770
8984
398857
32785
40497
5673
12377
491442
2025
889
16626
27782
2650
9174
1207
12751
1322
52842
704
139156
46367
4418
32313
2191
2997
15739
43451
4543
48872
2269
8197
13439
290
3637
5429
1924
10366
644
4738
7919
892
5411
1591
244
4663
54051
7309
5032
11619
14496
29000
38287
2635
128847
13011
9922
30323
7535
813
866149
5893
3817
1532
3811
1099
169037
5487
26438
4650
901054
182045
73927
6539
379304
137597
170569
3179
10357
11146
1482350
508557
3555
13567
20206
542
13413
1971
2627
14233
2105
558057
749
1212694
189509
4895
314172
357
1393
6760
2699
39508
963
44013
22825
40978
1975
20266
346882
73799
590395
44335
106092
10077
17497
13258
75866
8424
6679
844
31721
378
1344
4580
63443
99235
32366
14076
53348
2487
588
12649
1049
42768
785
13099
3490
8443
204111
11456
364942
94262
16214
33355
23832
33361
2428
1603
32434
2087
3913
11380
4897
48431
2103
72111
74771
3063
20831
4449
61288
10939
20336
6

26519
74167
77935
32014
75495
5699
2836
7163
18307
4561
11087
4461
30329
4291
7560
22750
9767
31946
37741
231163
1479
18286
27964
1411
32131
13882
2429
8181
14168
6026
365115
131280
19102
4150
834
45449
8923
3364
11464
24964
58851
3506
16827
8145
6574
60433
3037
504
13703
3503
392
19467
44584
13102
13234
1324
56534
14449
635598
422
417613
273750
124768
17772
31845
28022
1387
5454
51560
46539
40033
84897
3822
26782
1643
1220
42617
304233
177389
24880
4309
17055
80104
4922
44293
4978
15554
154989
8651
10157
147050
5431
21180
6732
147621
22097
37334
361705
4531
6312
23113
162724
1636
159614
1653
8323
4207
39312
81237
82250
897376
22020
24427
1618
7945
2605
30027
15349
136027
21496
30030
1312
67601
52503
271053
907
16236
9150
61634
171589
2838
1373
19380
6834
5517
88925
37294
8583
46918
41650
32061
4283
1663571
28069
25569
145097
88854
11787
841
56133
885090
195905
121984
2073
2849
15438
11179
6223
19274
35144
180060
47287
52097
2758
19926
22181
748326
292886
525611
1861
149759
411758
6021

4734
11831
32782
25776
1527
230604
178280
10124
113826
1308296
37975
655
14975
22389
1401
1985
5578
19922
148309
3385
17321
1484
46354
8665
2320
2075
4081
106726
3751
53757
54340
83434
34523
80099
145738
104985
36042
36884
39838
161454
16114
41053
17368
1408549
504653
31893
1478
42331
2453
249
926
30621
12177
51819
2912
436
88525
51818
158216
2074632
1321815
1891129
341585
48180
1716
80896
629
511851
3263
27809
90696
118182
172522
16015
1063883
379209
347453
30501
3672
125796
174052
3355
23625
10137
23733
3534
99032
10132
405126
4101
9498
8951
1873
4983
21655
95395
9775
414051
3457
4857
1164449
21664
15052
9125
4579
8217
1685398
16479
399085
33830
42201
11838
11835
3488
27607
2619
30848
13558
31182
13359
15486
6079
6874
735
9054
6247
8597
1221130
586811
7691
28756
26028
1832087
244410
42563
8046
62952
4070
350378
70527
6862
7677
564623
481
21966
98843
34719
9425
2188
16044
5485
79162
49769
3269
6517
7361
761
678
16435
2149
25542
3113
118080
3243
20766
29394
3285
13216
368868
1623
25935

119385
13279
29098
1039160
67383
86161
55567
22731
5023
73886
17678
7739
24399
7857
11580
12420
31167
29248
60968
4803
3494
5208
5999
11309
50197
2534
2962
6361
24467
376503
72333
12070
9060
138580
49687
1329
9302
69094
13610
3788
10516
15481
3341
13297
778691
19022
26284
60252
8902
33555
90974
54570
10833
405710
1348034
3237
56551
110219
6314
53076
19320
17515
77364
17874
29485
4270
1120852
469569
41662
99344
8569
17100
324057
2802
632
898763
54709
32540
36850
2965
39641
74891
62809
196463
56068
64075
93949
13243
533953
53567
43036
9349
36870
494809
2503
78262
128214
97803
51190
27050
5955
14677
6705
13992
10081
272940
24147
42659
27128
8618
100497
32767
160482
24586
81717
17135
82416
21125
27640
103382
448807
6974
532573
50478
52983
316419
39469
24822
3524
3605
36399
51592
281808
158719
243122
4765
6396
36131
813171
11885
2159
18405
12227
6172
42960
61244
174510
76864
131192
6454
17095
28796
20288
13708
109882
26413
21304
4817
3551
20802
2949
45763
62810
5254
5770
49745
4458
199424
8

...

652566
164895
430893
273390
46483
17076
46037
229248
22857
1768674
201333
614951
55225
342431
296649
142431
109940
100806
763619
103344
9207
902938
13803
258613
17379
64740
36420
1315840
28824
206776
482484
30012
33096
29017
51702
177989
23873
1628163
430528
99317
1667775
18059
784656
11152
46154
101444
32567
8774
51092
117618
1212435
35329
107068
15558
36645
196956
65713
1299473
150753
13475
18487
47520
266006
7904
79943
18420
696545
27945
29069
572378
44461
6916
1209441
555168
144731
21836
12155
672644
22419
387889
70228
94131
18966
103000
115570
12593
164719
12906
11360
9182
55130
17465
18123
8187
592295
73960
17665
2915
60105
667579
30371
54206
41209
225588
2066260
10906
24768
638000
66096
1047939
617431
37782
60244
203706
78632
10547
179374
9912
666350
60439
324227
102389
94004
24085
21929
117086
42174
36335
373119
205778
25786
20193
14206
7230
50095
4104
18555
33210
23488
12104
8781
36079
16849
234365
33042
200904
172091
592273
7084
213176
32905
121410
136497
389554
220887
748396

84189
232860
355916
495840
100960
76121
44472
155221
1671245
783783
76672
89235
8822
193448
56705
59469
347132
5629
727836
50404
77350
578950
15115
833315
83602
267514
583938
334865
41362
35909
50954
229886
72215
69267
12846
30952
119939
51806
20036
585948
394601
642703
191027
37092
5516
32348
21866
171149
6544
11185
538159
493801
14398
221511
38375
33354
8868
305620
37886
119758
99794
58816
170523
191179
12081
497473
803078
18687
1288020
28408
264064
73564
50660
4798
2230994
39784
31090
63988
66135
19296
259869
154757
15242
532225
41875
5303
44214
46189
1540132
1871820
44469
703291
35739
99086
3379
248224
19361
45440
7647
91020
778173
38050
167991
4955
108369
279569
83375
40967
125442
1753547
30401
6925
102841
26889
166275
36011
30559
29025
135905
290526
251092
15972
71448
263138
7171
24370
7577
468327
11297
37484
767726
73605
112537
119915
196874
682455
70103
50532
30908
103149
13412
30742
11932
40682
60416
89958
196666
20771
322702
2788
254257
19085
296826
119224
146443
4404
14709
5

132792
34623
120448
18729
89278
444983
103328
34192
28669
92000
38552
30883
4483
169960
67207
60174
164599
42870
1096531
40587
1343234
1207892
367668
215386
48709
48474
69127
17134
83195
32422
60583
631351
38032
28208
63023
1492439
192027
103152
24889
83028
136777
193444
18236
21602
138611
14681
34531
248191
7439
152364
1530687
18397
219118
1996908
1497947
380613
20222
1049861
38424
1606288
161694
845149
59511
33533
7892
560571
24527
685498
59146
5307
290099
118864
154031
153783
1254
25351
130748
82275
32479
144168
126019
66787
99455
37024
28404
30174
493959
126139
100538
57384
45505
114287
24932
23939
138246
12782
1062677
22352
258134
103267
31144
92755
50933
25783
146916
279369
81359
29726
86081
1448241
21529
23559
616862
34794
46085
84293
512988
47979
26528
9656
317354
33406
60972
14913
513190
1113495
246618
18120
7462
225093
142521
56382
356748
20596
82249
9376
50746
439377
44953
26901
30046
95344
22635
100396
471084
18372
610159
56256
192151
2028660
433717
39380
1804811
1480
12256

108072
46624
97801
55354
350091
1182405
28080
25810
2088952
540171
75860
55439
529592
26201
51021
61536
117032
62528
27527
14534
17825
36785
73099
127226
78548
24670
65941
5903
88553
192026
97672
343894
17239
1454670
18957
218528
28370
15837
34152
156012
65121
199607
73722
73934
46667
502364
84945
10311
133453
111082
20359
192428
31421
76219
41009
1531416
62117
613153
533550
204661
11975
8114
36790
32316
88686
52237
49797
92956
33495
191619
151526
164925
160879
156416
278623
21997
105409
371297
24133
36328
228886
179895
87948
120470
56931
93864
124932
1244176
3479
425781
24356
1843826
34806
120482
934205
110835
594829
39342
21833
415721
338956
201729
155608
98594
162812
154407
1029975
24592
64633
133618
12831
146168
180107
10565
5477
2081777
19064
56228
43012
68906
104986
152911
23140
673171
1805680
347256
6053
29874
72156
496659
274474
297820
232614
73639
697607
2245034
41701
18419
52152
2475
73135
10336
34352
361572
106619
1641582
54568
1018321
166500
66250
892079
38600
862485
172837

KeyboardInterrupt: 

In [11]:
test_labels = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_labels.npy')

In [12]:
print(test_labels[0])

0


In [19]:
output = set()
trends = [1,2,6,8,9,7,6,5,4,3,4,5,]
for x in trends:
    output.add(x)
print (output)

{1, 2, 3, 4, 5, 6, 7, 8, 9}


In [15]:
model_word2vec_temp.wv.syn0

array([[ 1.1291504e-03, -8.9645386e-04,  3.1852722e-04, ...,
        -1.5640259e-03, -1.2302399e-04, -8.6307526e-05],
       [ 7.0312500e-02,  8.6914062e-02,  8.7890625e-02, ...,
        -4.7607422e-02,  1.4465332e-02, -6.2500000e-02],
       [-1.1779785e-02, -4.7363281e-02,  4.4677734e-02, ...,
         7.1289062e-02, -3.4912109e-02,  2.4169922e-02],
       ...,
       [-1.9653320e-02, -9.0820312e-02, -1.9409180e-02, ...,
        -1.6357422e-02, -1.3427734e-02,  4.6630859e-02],
       [ 3.2714844e-02, -3.2226562e-02,  3.6132812e-02, ...,
        -8.8500977e-03,  2.6977539e-02,  1.9042969e-02],
       [ 4.5166016e-02, -4.5166016e-02, -3.9367676e-03, ...,
         7.9589844e-02,  7.2265625e-02,  1.3000488e-02]], dtype=float32)

In [ ]:
vocab_list = np.asarray(vocab_list)
np.save(save_path + save_name + '_word_indices' + '.npy', vocab_list)